In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys, json
import torch
import os
import numpy as np
import opennre
from opennre import encoder, model, framework

In [3]:
!pip list | grep opennre
import sys
sys.path

opennre                      0.1                 /Users/fenghaoguo/Downloads/github/OpenNRE


['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug',
 '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev',
 '/Users/fenghaoguo/code/git/fhaoguo/RelationExtraction',
 '/Users/fenghaoguo/code/git/fhaoguo/RelationExtraction',
 '/Users/fenghaoguo/opt/anaconda3/envs/p39/lib/python39.zip',
 '/Users/fenghaoguo/opt/anaconda3/envs/p39/lib/python3.9',
 '/Users/fenghaoguo/opt/anaconda3/envs/p39/lib/python3.9/lib-dynload',
 '',
 '/Users/fenghaoguo/.local/lib/python3.9/site-packages',
 '/Users/fenghaoguo/opt/anaconda3/envs/p39/lib/python3.9/site-packages',
 '/Users/fenghaoguo/Downloads/github/OpenNRE']

In [5]:
MAX_LEN = 10
with open("./data/people-relation/people-relation_train.txt","r") as f:
    index = 0
    while True:
        line = json.loads(f.readline())
        print(line)
        print("\n")
        print("".join(line['token']))
        print("<{},{},{}>".format(line['h']['name'],line['relation'],line['t']['name'] ))
        print("")
        print("-"*50)
        index += 1
        if index == MAX_LEN:
            break

{'token': ['后', '世', '解', '州', '、', '当', '阳', '、', '洛', '阳', '等', '地', '关', '姓', '氏', '族', '，', '常', '自', '称', '为', '关', '兴', '后', '裔', '，', '关', '羽', '-', '历', '史', '概', '括', '关', '兴', '蜀', '侍', '中', '、', '中', '监', '军', '。'], 'h': {'name': '关兴', 'pos': [21, 22]}, 't': {'name': '关羽', 'pos': [26, 27]}, 'relation': '父母'}


后世解州、当阳、洛阳等地关姓氏族，常自称为关兴后裔，关羽-历史概括关兴蜀侍中、中监军。
<关兴,父母,关羽>

--------------------------------------------------
{'token': ['郭', '台', '铭', '与', '二', '弟', '郭', '台', '强', '聽', '2', '0', '1', '0', '年', '六', '月', '十', '日', '，', '当', '台', '湾', '首', '富', '郭', '台', '铭', '正', '在', '为', '富', '士', '康', '深', '圳', '厂'], 'h': {'name': '郭台铭', 'pos': [0, 2]}, 't': {'name': '郭台强', 'pos': [6, 8]}, 'relation': '兄弟姐妹'}


郭台铭与二弟郭台强聽2010年六月十日，当台湾首富郭台铭正在为富士康深圳厂
<郭台铭,兄弟姐妹,郭台强>

--------------------------------------------------
{'token': ['年', '3', '月', '，', '国', '家', '体', '育', '总', '局', '举', '办', '庆', '祝', '“', '三', '鈥', '\ue76b', '恕', '惫', '\ue30b', '世', '投', '\ue1ed', '九', '\ue194', '诹', '\ue01

## Bert 预训练模型

1. 下载预训练的中文 
2. 放`OpenNER`的`./pretrain/`目录中

![](./files/Bert.png)
![](./files/Bert2.png)

In [6]:
root_path = "./"
# Check data
rel2id = json.load(open(os.path.join(root_path, 'data/people-relation/people-relation_rel2id.json')))

In [7]:
rel2id

{'父母': 0,
 '夫妻': 1,
 '师生': 2,
 '兄弟姐妹': 3,
 '合作': 4,
 '情侣': 5,
 '祖孙': 6,
 '好友': 7,
 '亲戚': 8,
 '同门': 9,
 '上下级': 10,
 'unknown': 11}


## Define the sentence encoder

In [8]:
os.path.join(root_path,'pretrain/chinese_wwm_pytorch')

'./pretrain/chinese_wwm_pytorch'

In [9]:
sentence_encoder = opennre.encoder.BERTEncoder(
    max_length=80, 
    pretrain_path=os.path.join(root_path,'pretrain/chinese_wwm_pytorch'),
    mask_entity='store_true'
)

2022-04-24 21:15:14,879 - root - INFO - Loading BERT pre-trained checkpoint.
Some weights of the model checkpoint at ./pretrain/chinese_wwm_pytorch were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Define the model

In [10]:
model = opennre.model.SoftmaxNN(sentence_encoder, len(rel2id), rel2id)
ckpt = './ckpt/v3/model.pth'

In [11]:
# Define the whole training framework
framework = opennre.framework.SentenceRE(
    train_path=os.path.join(root_path, 'data/people-relation/people-relation_train.txt'),
    val_path=os.path.join(root_path, 'data/people-relation/people-relation_val.txt'),
    test_path=os.path.join(root_path, 'data/people-relation/people-relation_val.txt'),
    model=model,
    ckpt=ckpt,
    batch_size=32, # Modify the batch size w.r.t. your device
    max_epoch=1,
    lr=2e-5,
    opt='adamw'
)

# Train the model
framework.train_model()

2022-04-24 21:20:40,200 - root - INFO - Loaded sentence RE dataset ./data/people-relation/people-relation_train.txt with 198000 lines and 12 relations.
2022-04-24 21:20:40,305 - root - INFO - Loaded sentence RE dataset ./data/people-relation/people-relation_val.txt with 1000 lines and 12 relations.
2022-04-24 21:20:40,426 - root - INFO - Loaded sentence RE dataset ./data/people-relation/people-relation_val.txt with 1000 lines and 12 relations.
/Users/fenghaoguo/opt/anaconda3/envs/p39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
2022-04-24 21:20:40,442 - root - INFO - === Epoch 0 train ===
  0%|          | 26/6188 [10:01<30:34:30, 17.86s/it, acc=0.132, loss=2.47]  Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x

Error in callback <bound method AutoreloadMagics.post_execute_hook of <IPython.extensions.autoreload.AutoreloadMagics object at 0x7fcfc6c11340>> (for post_execute):


KeyboardInterrupt: 

In [ ]:
# Test the model
framework.load_state_dict(torch.load(ckpt)['state_dict'])
result = framework.eval_model(framework.test_loader)

# Print the result
print('Accuracy on test set: {}'.format(result['acc']))